# Practicing the OpenSim Library - Building a model of a point mass.
# Tutorial from OpenSim youtube channel: https://www.youtube.com/watch?v=CPx0JzxIBKc&list=PLGBiefGDmSuR5NbZFuGOrjnTBx--H-H3C&index=3 
# This tutorial is the python version while the webinar used Matlab to script 

<!-- Importing and creating an empty model -->

In [1]:
# Importing OpenSim module/library
import opensim as osim

# Creating model instance from Model class
ptmass = osim.Model()

<!-- Creating body with parameters, adding geometry, and add body to the model -->

In [2]:
# Create a body with mass 1kg
# The arguments are: name, mass, CoM (0 means origin), and inertia matrix
body = osim.Body('body', 1.0, osim.Vec3(0), osim.Inertia(0))

# Creating geometry for the body - body does not habe any visualization
body.attachGeometry(osim.Sphere(0.1)) # Radius 0.1 m

# Adding the body to the model (ptmass)
ptmass.addBody(body)

<!-- Adding joint type and adding joint to the model -->

In [3]:
# Adding joint to the model (ptmass) - in this case slider joint
# The arguments are: name, parent body, and child body
sldr_jnt = osim.SliderJoint('sldr_jnt', ptmass.getGround(), body)
coord = sldr_jnt.updCoordinate()
coord.setName('translation')

# Adding the joint to the model (ptmass)
ptmass.addJoint(sldr_jnt)

<!-- Adding actuator (force) and adding force to the model -->

In [4]:
# Actuating (as Force) the model - we use CoordinateActuator. The argument is the name of the coordinate (translation)
actuator = osim.CoordinateActuator('translation')
actuator.setName('actuator') # setting the actuator name

# Adding the force the model (ptMass)
ptmass.addForce(actuator)

<!-- Adding prescribed controller and add it into the actuator. Prescribing the control to the actuator -->

In [5]:
# Adding the prescribed controller - apply controls that are a function of time only
controller = osim.PrescribedController()
controller.addActuator(actuator) # adding the coordinate actuator to the controller

# Prescribing the control to the actuator. The arguments are: name and the function
controller.prescribeControlForActuator('actuator', osim.Sine()) 

# Adding the controller to the model (ptmass)
ptmass.addController(controller)

<!-- Finalising the model and saving the model into .XML file -->

In [ ]:
# Creating and saving the model as an osim file (.osim)
ptmass.finalizeConnections()
ptmass.printToXML('ptmass_model.osim')

<!-- Conducting a Simulation. The steps are:
1-Enabling API visualizer window
2-Prepare the model for simulation. Initialise the internal "System" -->

In [7]:
# Simulate the model
ptmass.setUseVisualizer(True) # Enabling the visualizer
initState = ptmass.initSystem() # Initialising the internal "System"

<!-- OpenSim has two most important classes, Model (System) and State. The model (system) represents the mathematical equation and the state represents variables which are time-varying (coordinates, velocity, muscle states, etc). For more detail info, please check the webinar by clicking the link provided above -->

In [ ]:
# state object has the function getY, which is crucial
print(initState.getY()) # getY() returns a vector containing all state variables values
finalState = osim.simulate(ptmass, initState, 1.5)

<!-- Analysing a simulation
1-Get the final position of the point mass
2-Get the final CoM position of the point mass
3-Get the joint reaction moments and forces
For more detail of the explanation regarding the result printed, please check the tutorial video -->

In [13]:
print(coord.getValue(finalState)) # final position of the ptmass. Returns coordinate
ptmass.realizePosition(finalState)
print(ptmass.calcMassCenterPosition(finalState))
ptmass.realizeAcceleration(finalState)
print(sldr_jnt.calcReactionOnParentExpressedInGround(finalState))

0.5028553322349781
~[0.502855,0,0]
~[~[-0,0,-4.93133],~[-0.997495,-9.80665,-0]]
